In [74]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools

In [147]:
model = MultinomialNB()
vectorizer = CountVectorizer(ngram_range=(1, 2))
def naive_bayes(labels, examples):
    # Preparação dos dados de treinamento
    texts = []
    for idx, label in enumerate(labels):
        texts.extend(examples[idx])
    labels = [label for sublist in examples for label in [labels[examples.index(sublist)]] * len(sublist)]
    
    global vectorizer
    global model

    # Criação do vetorizador usando as palavras-chave dos equipamentos
    X_train = vectorizer.fit_transform(texts)
    # Treinamento do modelo
    model.fit(X_train, labels)
    
    # Fazer previsões nos dados de treinamento
    predicted_labels = model.predict(X_train)
    
    # Calcular a matriz de confusão
    confusion_mat = confusion_matrix(labels, predicted_labels)
    
    # Calcular a acurácia
    accuracy = np.trace(confusion_mat) / np.sum(confusion_mat)
    print("Acurácia: {:.2f}%".format(accuracy * 100))


In [18]:
from re import sub
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from spacy.lang.pt.stop_words import STOP_WORDS
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [35]:
def remove_num(text):
    text = sub(r'\d+', '', text)
    text = sub(r'\s+', ' ',text)
    return text

def remove_punct(text):
    text = sub(r"[!#$%&'()*+,-./:;<=>?@[^_`{|}~]+", ' ',text)
    text = sub(r'\s+', ' ',text)
    return text

def get_keywords(text):
    tokens = word_tokenize(text)
    keywords = []
    for word in tokens:
        word = word.lower()
        if word not in stopwords.words('portuguese') or word.lower() not in STOP_WORDS:
            keywords.append(word)
    return keywords

def get_synonyms(text):
    tokens = word_tokenize(text)
    synonyms = []
    for word in tokens:
        for syn in wordnet.synsets(word, lang="por"):
            for lemma in syn.lemmas(lang="por"):
                synonyms.append(lemma.name())
    return synonyms

def preprocess_lemma(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    lemmas = []
    for token in tokens:
        lemmas.append(lemmatizer.lemmatize(token))
    lemmas = ' '.join(lemmas)
    return lemmas

def remove_accent(text):
    text = sub('[áàãâä]', 'a', sub('[éèêë]', 'e', sub('[íìîï]', 'i', sub('[óòõôö]', 'o', sub('[úùûü]', 'u', text)))))
    text = sub(r'\s+', ' ',text)
    return text

def preprocess_stem(text):
    stemmer = SnowballStemmer("portuguese")
    tokens = word_tokenize(text)
    stems = []
    for token in tokens:
        stems.append(stemmer.stem(token))
    stems = ' '.join(stems)
    return stems

### Preparando o dataframe

In [20]:
import pandas as pd

In [21]:
df = pd.read_excel(r'..\documents\classes.xlsx')
df

,equipament,keywords
0,bombas,bomba
1,rolamentos,rolamento
2,válvulas,válvula
3,acionamentos por corrente,"corrente, acionamento corrente"
4,caixas de engrenagens,caixa engrenagem
5,Sistemas de óleo lubrificante,"reservatório óleo, bomba óleo, filtro óleo, ra..."
6,Acionamentos por correia em V,"polias V, correia V, tensionadores V, acoplame..."
7,Sistemas de ventiladores,"ventilador, exaustor, soprador"
8,Purgadores de vapor,Purgadores vapor
9,Motores elétricos,Motores elétricos


In [153]:
bombas = [
    "Bomba centrífuga",
    "Bomba de água",
    "Bomba de vácuo",
    "Bomba de calor",
    "Bomba dosadora",
    "Bomba submersível",
    "Bomba de pistão",
    "Bomba de diafragma",
    "Bomba de engrenagem",
    "Bomba de incêndio",
    "Bomba de vácuo rotativa",
    "Bomba de alta pressão",
    "Bomba de transferência",
    "Bomba de ar comprimido",
    "Bomba de esgoto",
    "Bomba peristáltica",
    "Bomba de irrigação",
    "Bomba de deslocamento positivo",
    "Bomba de lóbulos",
    "Bomba de turbina",
    "Bomba de palhetas",
    "Bomba de cavidade progressiva",
    "Bomba hidráulica",
    "Bomba de duplo diafragma",
    "Bomba de poço",
    "Bomba de drenagem",
    "Bomba de concreto",
    "Bomba de óleo",
    "Bomba de químicos",
    "Bomba de refrigeração"
]
rolamentos = [
    "Rolamento de esferas",
    "Rolamento de rolos",
    "Rolamento axial",
    "Rolamento radial",
    "Rolamento de agulhas",
    "Rolamento autocompensador",
    "Rolamento de contato angular",
    "Rolamento de precisão",
    "Rolamento magnético",
    "Rolamento cerâmico",
    "Rolamento blindado",
    "Rolamento vedado",
    "Rolamento linear",
    "Rolamento de alta temperatura",
    "Rolamento de baixo atrito",
    "Rolamento de longa vida útil",
    "Rolamento de elevada capacidade de carga",
    "Rolamento de alta velocidade",
    "Rolamento de rolamento axial de rolos cilíndricos",
    "Rolamento de rolo cilíndrico",
    "Rolamento de esferas de contato angular de quatro pontos",
    "Rolamento de rolo esférico",
    "Rolamento de giro",
    "Rolamento para máquinas pesadas",
    "Rolamento para alta precisão",
    "Rolamento para altas temperaturas",
    "Rolamento para baixo ruído",
    "Rolamento para aplicações industriais",
    "Rolamento de transmissão de potência",
    "Rolamento de movimentação linear"
]
valvulas = [
    "Válvula de controle",
    "Válvula de alívio",
    "Válvula solenóide",
    "Válvula de esfera",
    "Válvula de borboleta",
    "Válvula de retenção",
    "Válvula de gaveta",
    "Válvula de diafragma",
    "Válvula de agulha",
    "Válvula de segurança",
    "Válvula de escape",
    "Válvula de bloqueio",
    "Válvula reguladora de pressão",
    "Válvula de descarga",
    "Válvula de enchimento",
    "Válvula de descarga a vácuo",
    "Válvula de reabastecimento",
    "Válvula de admissão",
    "Válvula de exaustão",
    "Válvula de pressão diferencial",
    "Válvula de controle de vazão",
    "Válvula de alívio de pressão",
    "Válvula de retenção de pé",
    "Válvula de alívio térmico",
    "Válvula de retenção de disco duplo",
    "Válvula de controle direcional",
    "Válvula de retenção de pistão",
    "Válvula de segurança de pressão",
    "Válvula de controle proporcional",
    "Válvula de isolamento"
]
acionamentos_corrente = [
    "Acionamento por corrente",
    "Motor de acionamento por corrente",
    "Acionamento por corrente elétrica",
    "Corrente de acionamento",
    "Sistema de acionamento por corrente",
    "Acionamento por corrente contínua",
    "Acionamento por corrente alternada",
    "Corrente de acionamento elétrico",
    "Acionamento por corrente trifásica",
    "Acionamento por corrente monofásica",
    "Acionamento por corrente direta",
    "Acionamento por corrente reversa",
    "Acionamento por corrente pulsada",
    "Acionamento por corrente de alta tensão",
    "Acionamento por corrente de baixa tensão",
    "Acionamento por corrente constante",
    "Corrente de acionamento ajustável",
    "Acionamento por corrente variável",
    "Acionamento por corrente programável",
    "Corrente de acionamento estável",
    "Acionamento por corrente de precisão",
    "Acionamento por corrente de alta potência",
    "Acionamento por corrente de baixa potência",
    "Acionamento por corrente de média potência",
    "Corrente de acionamento industrial",
    "Acionamento por corrente de tração",
    "Acionamento por corrente de transporte",
    "Acionamento por corrente de elevação",
    "Corrente de acionamento automático",
    "Acionamento por corrente de controle",
]
caixas_engrenagens = [
    "Caixa de engrenagens",
    "Engrenagens industriais",
    "Sistema de transmissão por engrenagens",
    "Caixa de engrenagens helicoidais",
    "Caixa de engrenagens cônicas",
    "Engrenagens de alta precisão",
    "Caixa de engrenagens de redução",
    "Engrenagens de grande torque",
    "Caixa de engrenagens de alta potência",
    "Engrenagens de baixo ruído",
    "Caixa de engrenagens de velocidade variável",
    "Engrenagens de transmissão",
    "Caixa de engrenagens de eixos paralelos",
    "Engrenagens de dentes retos",
    "Caixa de engrenagens planetárias",
    "Engrenagens de eixos cruzados",
    "Caixa de engrenagens de precisão",
    "Engrenagens de múltiplas velocidades",
    "Caixa de engrenagens de montagem fácil",
    "Engrenagens de longa vida útil",
    "Caixa de engrenagens de operação suave",
    "Engrenagens de alto desempenho",
    "Caixa de engrenagens de serviço pesado",
    "Engrenagens de reversão",
    "Caixa de engrenagens de transmissão de potência",
    "Engrenagens de lubrificação permanente",
    "Caixa de engrenagens de carga elevada",
    "Engrenagens de troca rápida",
    "Caixa de engrenagens de operação silenciosa",
    "Engrenagens de baixa manutenção",
]
sistemas_oleo_lubrificante = [
    "Sistema de óleo lubrificante",
    "Bomba de óleo",
    "Filtro de óleo",
    "Radiador de óleo",
    "Trocador de óleo",
    "Válvula de óleo",
    "Sensor de óleo",
    "Tubulação de óleo",
    "Respiro de óleo",
    "Reservatório de óleo",
    "Óleo lubrificante",
    "Sistema de lubrificação",
    "Fluido lubrificante",
    "Viscosidade do óleo",
    "Análise de óleo",
    "Nível de óleo",
    "Troca de óleo",
    "Contaminação do óleo",
    "Viscosímetro de óleo",
    "Bomba de circulação de óleo",
    "Monitoramento do óleo",
    "Tratamento do óleo",
    "Descarte de óleo",
    "Amostragem de óleo",
    "Sistema de filtragem de óleo",
    "Bomba de transferência de óleo",
    "Válvula de alívio de pressão do óleo",
    "Fluxo de óleo",
    "Sistema de resfriamento de óleo",
    "Sistema de aquecimento de óleo",
]
acionamentos_correia_v = [
    "Acionamentos por correia em V",
    "Polias em V",
    "Correias em V",
    "Tensionadores em V",
    "Acoplamentos em V",
    "Esticadores em V",
    "Redutores em V",
    "Suportes em V",
    "Sistemas de transmissão em V",
    "Mecanismos de acionamento em V",
    "Polias e correias em V",
    "Sincronização em V",
    "Ajuste de correias em V",
    "Troca de correias em V",
    "Manutenção de acionamentos em V",
    "Desgaste de correias em V",
    "Alinhamento de correias em V",
    "Tensão das correias em V",
    "Substituição de correias em V",
    "Monitoramento de acionamentos em V",
    "Vida útil das correias em V",
    "Eficiência dos acionamentos em V",
    "Transmissão de potência em V",
    "Ruído nos acionamentos em V",
    "Vibração nos acionamentos em V",
    "Inspeção dos acionamentos em V",
    "Lubrificação dos acionamentos em V",
    "Falha nas correias em V",
    "Desalinhamento dos acionamentos em V",
    "Desbalanceamento dos acionamentos em V",
]
sistemas_ventiladores = [
    "Sistemas de ventiladores",
    "Ventiladores industriais",
    "Exaustores",
    "Sopradores",
    "Ventilação industrial",
    "Manutenção de ventiladores",
    "Limpeza de ventiladores",
    "Inspeção de ventiladores",
    "Eficiência dos ventiladores",
    "Controle de velocidade dos ventiladores",
    "Ruído dos ventiladores",
    "Vibração dos ventiladores",
    "Monitoramento dos ventiladores",
    "Fluxo de ar dos ventiladores",
    "Pressão dos ventiladores",
    "Temperatura dos ventiladores",
    "Motor dos ventiladores",
    "Filtragem do ar dos ventiladores",
    "Troca de filtros dos ventiladores",
    "Dutos de ar dos ventiladores",
    "Sistema de exaustão",
    "Sistema de insuflamento de ar",
    "Sistema de ventilação industrial",
    "Seleção de ventiladores",
    "Dimensionamento de ventiladores",
    "Curvas de desempenho dos ventiladores",
    "Mancais dos ventiladores",
    "Lubrificação dos ventiladores",
    "Falha nos ventiladores",
    "Substituição de ventiladores"
]
purgadores_vapor = [
    "Purgadores de vapor",
    "Sistema de purga de vapor",
    "Condensado de vapor",
    "Descarga de condensado",
    "Válvulas de purga de vapor",
    "Dreno de vapor",
    "Manutenção de purgadores de vapor",
    "Limpeza de purgadores de vapor",
    "Inspeção de purgadores de vapor",
    "Eficiência dos purgadores de vapor",
    "Perda de vapor",
    "Vazamento de vapor",
    "Monitoramento de purgadores de vapor",
    "Temperatura do condensado de vapor",
    "Pressão do condensado de vapor",
    "Nível de condensado de vapor",
    "Fluxo de condensado de vapor",
    "Troca de purgadores de vapor",
    "Sistema de retorno de condensado",
    "Sistema de distribuição de vapor",
    "Sistema de condensação de vapor",
    "Dimensionamento de purgadores de vapor",
    "Seleção de purgadores de vapor",
    "Curvas de desempenho de purgadores de vapor",
    "Mancais de purgadores de vapor",
    "Lubrificação de purgadores de vapor",
    "Falha nos purgadores de vapor",
    "Substituição de purgadores de vapor",
    "Recuperação de energia de vapor",
    "Tratamento de condensado de vapor"
]
motores_eletricos = [
    "Motores elétricos",
    "Motor de indução",
    "Motor de corrente alternada",
    "Motor de corrente contínua",
    "Motor trifásico",
    "Motor monofásico",
    "Motor assíncrono",
    "Motor síncrono",
    "Partida de motores",
    "Parada de motores",
    "Manutenção de motores elétricos",
    "Limpeza de motores elétricos",
    "Inspeção de motores elétricos",
    "Eficiência dos motores elétricos",
    "Monitoramento de motores elétricos",
    "Ruído dos motores elétricos",
    "Vibração dos motores elétricos",
    "Temperatura dos motores elétricos",
    "Corrente dos motores elétricos",
    "Tensão dos motores elétricos",
    "Potência dos motores elétricos",
    "Enrolamentos dos motores elétricos",
    "Isolamento dos motores elétricos",
    "Falha nos motores elétricos",
    "Substituição de motores elétricos",
    "Rebobinamento de motores elétricos",
    "Sobrecarga de motores elétricos",
    "Proteção de motores elétricos",
    "Eficiência energética dos motores elétricos",
    "Vida útil dos motores elétricos"
]
# words = []
# for i in (bombas + rolamentos + valvulas + acionamentos_corrente + caixas_engrenagens + sistemas_oleo_lubrificante + acionamentos_correia_v + sistemas_ventiladores + purgadores_vapor + motores_eletricos):
#     i = ' '.join(get_keywords(i))
#     words.append(remove_accent(preprocess_stem(remove_num(i))).lower())
# len(words)
words = [bombas,rolamentos,valvulas,acionamentos_corrente,caixas_engrenagens,sistemas_oleo_lubrificante,acionamentos_correia_v,sistemas_ventiladores,purgadores_vapor,motores_eletricos]
for i in range(len(words)):
    for j in range(len(words[i])):
        x = ' '.join(get_keywords(remove_num(words[i][j])))
        words[i][j] = remove_accent(preprocess_stem(x)).lower()
words

[['bomb centrifug',
  'bomb agu',
  'bomb vacu',
  'bomb calor',
  'bomb dosador',
  'bomb submers',
  'bomb pista',
  'bomb diafragm',
  'bomb engrenag',
  'bomb incendi',
  'bomb vacu rotat',
  'bomb alta pressa',
  'bomb transferent',
  'bomb ar comprim',
  'bomb esgot',
  'bomb peristalt',
  'bomb irrig',
  'bomb desloc posit',
  'bomb lobul',
  'bomb turbin',
  'bomb palhet',
  'bomb cavidad progress',
  'bomb hidraul',
  'bomb dupl diafragm',
  'bomb poc',
  'bomb drenag',
  'bomb concret',
  'bomb ole',
  'bomb quimic',
  'bomb refriger'],
 ['rolament esfer',
  'rolament rol',
  'rolament axial',
  'rolament radial',
  'rolament agulh',
  'rolament autocompens',
  'rolament contat angul',
  'rolament precisa',
  'rolament magnet',
  'rolament ceram',
  'rolament blind',
  'rolament ved',
  'rolament lin',
  'rolament alta temperatur',
  'rolament baix atrit',
  'rolament long vid util',
  'rolament elev capac carg',
  'rolament alta veloc',
  'rolament rolament axial rol cilindr

In [170]:
dict_ = {}
for idx, row in df.iterrows():
    classe =  remove_accent(preprocess_stem(remove_num(row['equipament']))).lower()
    keywords = remove_accent(preprocess_stem(remove_num(row['keywords']))).lower()
    dict_[classe] = keywords.split(',')
lista_classes = [i[0] for i in list(enumerate(dict_.keys()))]

In [182]:
df_teste = pd.read_excel(r'..\documents\teste_modelo.xlsx')
df_teste.head()

,classe,exemplos
0,1,- A bomba está fazendo barulho excessivo e vib...
1,1,- A bomba não está ligando quando o sistema é ...
2,1,- A bomba está apresentando vazamento de água.
3,1,- A bomba está funcionando com baixo desempenho.
4,1,- A bomba está superaquecendo e desligando aut...


In [186]:
def classify_word(word):
    global vectorizer
    global model
    
    # Vetorizar a palavra usando o vetorizador global
    X_test = vectorizer.transform([word])
    
    # Fazer a previsão usando o modelo treinado
    predicted_label = model.predict(X_test)[0]
    
    return predicted_label

# Chamar a função naive_bayes para treinar o modelo e ajustar o vetorizador
naive_bayes(lista_classes, words)

acertos = 0
erros = 0
for idx, row in df_teste.iterrows():
    word = row['exemplos']
    predicted_label = classify_word(word)
    print("Palavra:", word)
    print("Classe:", row["classe"] - 1)
    print("Classificação prevista:", predicted_label)
    print()
    if predicted_label == row["classe"] - 1:
        acertos += 1
    else:
        erros += 1
print('acertos:', acertos)
print('erros:', erros)
print(acertos * 100/(acertos+erros),r'% de precisão')

Acurácia: 99.00%
Palavra: - A bomba está fazendo barulho excessivo e vibrando muito.
Classe: 0
Classificação prevista: 0

Palavra: - A bomba não está ligando quando o sistema é ativado.
Classe: 0
Classificação prevista: 0

Palavra: - A bomba está apresentando vazamento de água.
Classe: 0
Classificação prevista: 0

Palavra: - A bomba está funcionando com baixo desempenho.
Classe: 0
Classificação prevista: 0

Palavra: - A bomba está superaquecendo e desligando automaticamente.
Classe: 0
Classificação prevista: 0

Palavra: - O rolamento está emitindo ruídos estranhos quando girado.
Classe: 1
Classificação prevista: 0

Palavra: - O rolamento está travando, impedindo o movimento suave da máquina.
Classe: 1
Classificação prevista: 0

Palavra: - O rolamento está apresentando folga excessiva.
Classe: 1
Classificação prevista: 0

Palavra: - O rolamento está superaquecendo, indicando possível falta de lubrificação.
Classe: 1
Classificação prevista: 0

Palavra: - O rolamento está desgastado e pre

In [ ]:
from fuzzywuzzy import fuzz
from nltk.tokenize import word_tokenize

def similares(frase, lista_classes):
    palavras_similares = []
    palavras_frase = word_tokenize(frase.lower())

    for palavra in palavras_frase:
        similaridades = []
        for classe in lista_classes:
            similaridade = fuzz.ratio(palavra, classe)
            similaridades.append((classe, similaridade))

        similaridades = sorted(similaridades, key=lambda x: x[1], reverse=True)
        if similaridades and similaridades[0][1] >= 80:
            palavra_similar = {
                "palavra": palavra,
                "classe": similaridades[0][0],
                "similaridade": similaridades[0][1]
            }
        else:
            palavra_similar = {
                "palavra": palavra,
                "classe": None,
                "similaridade": 0
            }
        
        palavras_similares.append(palavra_similar)

    return palavras_similares
